In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv
/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv


In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')
from sklearn import *

[nltk_data] Downloading package stopwords to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_train=pd.read_csv("train.csv")
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
def clear_tweets(text):
    text=re.sub('<[^>]*>','',text)
    emojis=re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text=re.sub('[\W]+',' ',text.lower()) +' '.join(emojis).replace('-','')

    return text   

In [4]:
df_train['tweet']=df_train['tweet'].apply(clear_tweets)
df_train.head()

,id,label,tweet
0,1,0,user when a father is dysfunctional and is so...
1,2,0,user user thanks for lyft credit i can t use ...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in ur...
4,5,0,factsguide society now motivation


In [5]:
df_train['label'].unique()

array([0, 1], dtype=int64)

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop=stopwords.words('english')

stop_words = set(stopwords.words("english"))


In [7]:
stemmer = PorterStemmer()

In [8]:
def func_2(tweets):
    token=word_tokenize(tweets)
    filtered_list = []
    for word in token:
        if word.casefold() not in stop_words:
            filtered_list.append(word)
    stemmed_words = [stemmer.stem(word) for word in filtered_list]
    return stemmed_words

In [10]:
import nltk
nltk.download('punkt')
new_list=[]
arr=[]
i=0
for tweet in df_train['tweet']:
    new_list=func_2(tweet)
    joined_str=""
    for word in new_list:
        joined_str=joined_str+" "+word
    arr.append(joined_str)

[nltk_data] Downloading package punkt to C:\Users\Priyanshi
[nltk_data]     Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [11]:
df_train['new_tweet']=arr

In [12]:
df_train

,id,label,tweet,new_tweet
0,1,0,user when a father is dysfunctional and is so...,user father dysfunct selfish drag kid dysfunc...
1,2,0,user user thanks for lyft credit i can t use ...,user user thank lyft credit use caus offer wh...
2,3,0,bihday your majesty,bihday majesti
3,4,0,model i love u take with u all the time in ur...,model love u take u time urð ð ð ð ð ð ð ð
4,5,0,factsguide society now motivation,factsguid societi motiv
...,...,...,...,...
31957,31958,0,ate user isz that youuu ð ð ð ð ð ð ð ð ð â ï,ate user isz youuu ð ð ð ð ð ð ð ð ð â ï
31958,31959,0,to see nina turner on the airwaves trying to ...,see nina turner airwav tri wrap mantl genuin ...
31959,31960,0,listening to sad songs on a monday morning otw...,listen sad song monday morn otw work sad
31960,31961,1,user sikh temple vandalised in in calgary wso...,user sikh templ vandalis calgari wso condemn act


In [13]:
X_neg=[]
X_pos=[]
Y_pos=[]
Y_neg=[]
for t in df_train['new_tweet'][df_train['label']==0]:
    X_neg.append(t)
    
for t in df_train['label'][df_train['label']==0]:
    Y_neg.append(t)

for t in df_train['new_tweet'][df_train['label']==1]:
    X_pos.append(t)
    
for t in df_train['label'][df_train['label']==1]:
    Y_pos.append(t)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000)

X_vec=vectorizer.fit_transform(df_train.new_tweet)

In [29]:
y=df_train.label.values
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_vec,y,random_state=1,test_size=0.25,shuffle=False)

In [16]:
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.6min finished


Accuracy: 0.9628331873357527


In [102]:
df_test=pd.read_csv("test.csv")
test=df_test['tweet']
test=test.apply(clear_tweets)
test=test.apply(func_2)



arr=[]

for tweet in test:

    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)

test=pd.DataFrame(arr,columns=['tweet'])


test.head()


,tweet
0,studiolif aislif requir passion dedic willpow...
1,user white supremacist want everyon see new â...
2,safe way heal acn altwaystoh healthi heal
3,hp curs child book reserv alreadi ye ð ð ð ha...
4,3rd bihday amaz hilari nephew eli ahmir uncl ...


In [53]:
xtest=vectorizer.fit_transform(test.tweet)
xtest.shape
ytest=clf.predict(xtest)


In [54]:
final=pd.DataFrame(df_test['tweet'],columns=['tweet'])
final['label']=ytest
final.head()

,tweet,label
0,#studiolife #aislife #requires #passion #dedic...,0
1,@user #white #supremacists want everyone to s...,0
2,safe ways to heal your #acne!! #altwaystohe...,0
3,is the hp and the cursed child book up for res...,0
4,"3rd #bihday to my amazing, hilarious #nephew...",0


In [103]:
data=['Congratulations to His Excellency Ebrahim Raisi on his election as President of the Islamic Republic of Iran',
      ' I look forward to working with him to further strengthen the warm ties between India and Iran.',
      'Baghor shrine is a modest temple that only has a stone platform under Neem tree and a triangular shaped Yantra worshiped as Goddess in the middle of platform.',
      'Yet, it has been worshiped continuously since 11,000 years from Paleolithic hunter gatherers to modern day inhabitants',
      'This video legitimately made me wanna try Takis, and then the comment section talked me completely out of it.',
      'i am living',
      'guys for some reason i am in a minecraft tournament i am gonna be streaming it in like an hour',
      'astronaut in the ocean was written and performed by a youth pastor i’m sure of it']

In [104]:
data=pd.DataFrame(data,columns=['tweet'])
data.head()


,tweet
0,Congratulations to His Excellency Ebrahim Rais...
1,I look forward to working with him to further...
2,Baghor shrine is a modest temple that only has...
3,"Yet, it has been worshiped continuously since ..."
4,This video legitimately made me wanna try Taki...


In [105]:

text=data['tweet'].apply(clear_tweets)
text=text.apply(func_2)

arr=[]
for tweet in text:
    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)
    
text=pd.DataFrame(arr,columns=['tweet'])
test.shape

test=test.append(text,ignore_index=True)
test.tail(10)



,tweet
17195,happi work confer right mindset lead cultur d...
17196,song glad free download shoegaz newmus newsong
17197,congratul excel ebrahim raisi elect presid is...
17198,look forward work strengthen warm tie india iran
17199,baghor shrine modest templ stone platform nee...
17200,yet worship continu sinc 11 000 year paleolit...
17201,video legitim made wan na tri taki comment se...
17202,live
17203,guy reason minecraft tournament gon na stream...
17204,astronaut ocean written perform youth pastor ...


In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec= TfidfVectorizer(max_features=10000)  

from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
xval=vec.fit_transform(test.tweet)

yval=clf.predict(xval)

final=pd.DataFrame(test['tweet'],columns=['tweet'])
final['label']=yval
final.head()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.1min finished


,tweet,label
0,studiolif aislif requir passion dedic willpow...,0
1,user white supremacist want everyon see new â...,0
2,safe way heal acn altwaystoh healthi heal,0
3,hp curs child book reserv alreadi ye ð ð ð ha...,0
4,3rd bihday amaz hilari nephew eli ahmir uncl ...,0


In [107]:
(final['label']).value_counts()
final.head()

,tweet,label
0,studiolif aislif requir passion dedic willpow...,0
1,user white supremacist want everyon see new â...,0
2,safe way heal acn altwaystoh healthi heal,0
3,hp curs child book reserv alreadi ye ð ð ð ha...,0
4,3rd bihday amaz hilari nephew eli ahmir uncl ...,0


In [108]:
final.tail(20)

,tweet,label
17185,today good day excercis imreadi sofuckenreadi...,0
17186,good night tea music â ï ð ð ð billi music te...,0
17187,love lifeð ºð â ï ð createyourfutur lifestyl ...,0
17188,black professor demon propos nazi style confi...,0
17189,learn think posit posit instagram instagood,0
17190,love pretti happi fresh teenilici fixdermatee...,0
17191,2_damn_tuff ruff_muff__techno_c ng005 web 199...,0
17192,thought factori left right polaris trump usel...,0
17193,feel like mermaid ð hairflip neverreadi forma...,0
17194,hillari campaign today ohio omg amp use word ...,0


In [109]:
final['label'].value_counts()

0    16900
1      305
Name: label, dtype: int64

In [119]:
tweet=['You are beautiful love ']
data=pd.DataFrame(tweet,columns=['tweet'])
data.head()
text=data['tweet'].apply(clear_tweets)
text=text.apply(func_2)

arr=[]
for tweet in text:
    joined_str=""
    for word in tweet:
        joined_str=joined_str+" "+word
    arr.append(joined_str)
    
text=pd.DataFrame(arr,columns=['tweet'])
# test.shape

test=test.append(text,ignore_index=True)
# test.tail(10)
vec= TfidfVectorizer(max_features=10000)
clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
xval=vec.fit_transform(test.tweet)

yval=clf.predict(xval)

final=pd.DataFrame(test['tweet'],columns=['tweet'])
final['label']=yval
final.head()


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.1min finished


,tweet,label
0,studiolif aislif requir passion dedic willpow...,0
1,user white supremacist want everyon see new â...,0
2,safe way heal acn altwaystoh healthi heal,0
3,hp curs child book reserv alreadi ye ð ð ð ha...,0
4,3rd bihday amaz hilari nephew eli ahmir uncl ...,0


In [120]:
final['label'].tail(1)

17208    0
Name: label, dtype: int64

In [121]:
import pickle
data={"model":clf,"vectoriser":vec}
with open ('saved_steps.pk1','wb') as file:
    pickle.dump(data,file)

In [122]:
with open('saved_steps.pk1','rb') as file:
    data=pickle.load(file)
model_loaded=data['model']
vectoriser_loaded=data['vectoriser']


In [127]:
def predict(tweet,test):
    data=pd.DataFrame(tweet,columns=['tweet'])
#     data.head()
    text=data['tweet'].apply(clear_tweets)
    text=text.apply(func_2)

    arr=[]
    for tweet in text:
        joined_str=""
        for word in tweet:
            joined_str=joined_str+" "+word
        arr.append(joined_str)

    text=pd.DataFrame(arr,columns=['tweet'])
    # test.shape

    test=test.append(text,ignore_index=True)
    # test.tail(10)
    vec= TfidfVectorizer(max_features=10000)
    clf=LogisticRegressionCV(cv=6,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=500).fit(X_train,y_train)
    xval=vec.fit_transform(test.tweet)

    yval=clf.predict(xval)

    final=pd.DataFrame(test['tweet'],columns=['tweet'])
    final['label']=yval
    return final
#     final.head()

In [128]:
tweet=['You are beautiful love ']
final=predict(tweet,test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   52.5s remaining:   52.5s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   53.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   53.0s finished


In [138]:
final.tail(1)

,tweet,label
17209,beauti love,0


In [148]:
i=final.label.tail(1).values[0]
print(i)

0


In [141]:
print(type(i))

<class 'pandas.core.series.Series'>
